In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

train_data = pd.read_csv(r'C:\Users\HP\Downloads\train.csv')
test_data = pd.read_csv(r'C:\Users\HP\Downloads\test.csv')
submission_data = pd.read_csv(r'C:\Users\HP\Downloads\gender_submission.csv')


train_data



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
test_data

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,True,True,False
1,893,3,47.0,1,0,7.0000,False,False,True
2,894,2,62.0,0,0,9.6875,True,True,False
3,895,3,27.0,0,0,8.6625,True,False,True
4,896,3,22.0,1,1,12.2875,False,False,True
...,...,...,...,...,...,...,...,...,...
413,1305,3,27.0,0,0,8.0500,True,False,True
414,1306,1,39.0,0,0,108.9000,False,False,False
415,1307,3,38.5,0,0,7.2500,True,False,True
416,1308,3,27.0,0,0,8.0500,True,False,True


In [9]:
def preprocess_data(df):
    # Impute missing values for 'Age' and 'Embarked'
    age_imputer = SimpleImputer(strategy='median')
    df['Age'] = age_imputer.fit_transform(df[['Age']]).ravel()
    
    embarked_imputer = SimpleImputer(strategy='most_frequent')
    df['Embarked'] = embarked_imputer.fit_transform(df[['Embarked']]).ravel()
    
    # One-hot encode 'Sex' and 'Embarked' columns
    df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)
    
    return df

# Preprocess the training and test data
train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

# Drop unnecessary columns
train_data = train_data.drop(columns=['Cabin', 'Ticket', 'Name'])
test_data = test_data.drop(columns=['Cabin', 'Ticket', 'Name'])

# Define features and target variable
X = train_data.drop(columns=['Survived'])
y = train_data['Survived']

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a preprocessing pipeline
numerical_features = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features)
    ])

# Function to train and evaluate a model
def train_evaluate_model(model, param_grid):
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='precision')
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_val)
    
    precision = precision_score(y_val, predictions)
    recall = recall_score(y_val, predictions)
    
    return best_model, precision, recall

# Train and evaluate Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20]
}
best_rf_model, rf_precision, rf_recall = train_evaluate_model(rf_model, rf_param_grid)

# Train and evaluate SVM model
svm_model = SVC(random_state=42)
svm_param_grid = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['linear', 'rbf']
}
best_svm_model, svm_precision, svm_recall = train_evaluate_model(svm_model, svm_param_grid)

# Display precision and recall for both models
print("Random Forest - Precision:", rf_precision, "Recall:", rf_recall)
print("SVM - Precision:", svm_precision, "Recall:", svm_recall)

Random Forest - Precision: 0.7959183673469388 Recall: 0.527027027027027
SVM - Precision: 0.8918918918918919 Recall: 0.44594594594594594
